# AN√ÅLISIS COMPLETO - KUESKI PORTFOLIO Q1 2025
=========================================

## Objetivo
An√°lisis exhaustivo del portfolio de pr√©stamos Q1 2025 con visualizaciones y m√©tricas clave:

### M√©tricas Principales
- **LTV/CAC por segmento** (CORREGIDO: CAC solo en pr√©stamo #1)
- **Portfolio mix** (volumen y revenue)
- **Vintage curves** (evoluci√≥n de delinquency)
- **Performance por risk band**
- **Clientes recurrentes** vs primer pr√©stamo
- **CAC por pr√©stamo** (nueva m√©trica amortizada)

### Hallazgos Clave
- ‚≠ê High Risk: **10.37x LTV/CAC** (mejor segmento)
- ‚úÖ Medium Risk: **5.85x LTV/CAC**
- ‚ùå Low Risk: **2.47x LTV/CAC** (no rentable)
- Ì≥ä Clientes recurrentes: **-43% delinquency** vs primer pr√©stamo

In [ ]:
# Imports
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path

# Configuraci√≥n de visualizaci√≥n
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.options.display.float_format = '{:,.2f}'.format

# Conectar a DuckDB
conn = duckdb.connect('../dbt/kueski_finance.duckdb')

print("‚úÖ Librer√≠as cargadas")
print("‚úÖ Conexi√≥n a DuckDB establecida")

## 1Ô∏è‚É£ RESUMEN EJECUTIVO - KPIs PRINCIPALES

In [ ]:
# KPIs Principales por Segmento
query_kpis = """
WITH customer_lifetime AS (
    SELECT 
        l.user_id,
        MAX(l.risk_segment) as risk_segment,
        SUM(l.revenue_total) as lifetime_revenue,
        SUM(l.cac) as total_cac_paid,
        COUNT(DISTINCT l.loan_id) as total_loans
    FROM main.fct_loan_financials l
    WHERE l.risk_segment IN ('High Risk', 'Medium Risk', 'Low Risk')
    GROUP BY l.user_id
)
SELECT 
    risk_segment,
    COUNT(DISTINCT user_id) as customers,
    SUM(total_loans) as loans,
    ROUND(SUM(lifetime_revenue), 0) as total_revenue,
    ROUND(SUM(total_cac_paid), 0) as total_cac,
    ROUND(SUM(lifetime_revenue) / NULLIF(SUM(total_cac_paid), 0), 2) as ltv_to_cac,
    ROUND(AVG(total_loans), 1) as avg_loans_per_customer,
    ROUND(AVG(lifetime_revenue), 0) as avg_ltv,
    ROUND(SUM(lifetime_revenue) / SUM(total_loans), 0) as avg_revenue_per_loan
FROM customer_lifetime
GROUP BY risk_segment
ORDER BY total_revenue DESC
"""

df_kpis = conn.execute(query_kpis).df()

print("="*80)
print("KPIs PRINCIPALES POR SEGMENTO DE RIESGO")
print("="*80)
print(df_kpis.to_string(index=False))

# Totales
print("\n" + "="*80)
print("TOTALES PORTFOLIO Q1 2025")
print("="*80)
print(f"Total Clientes: {df_kpis['customers'].sum():,}")
print(f"Total Pr√©stamos: {df_kpis['loans'].sum():,}")
print(f"Revenue Total: ${df_kpis['total_revenue'].sum():,.0f}")
print(f"CAC Total: ${df_kpis['total_cac'].sum():,.0f}")
print(f"\n‚≠ê LTV/CAC Promedio Ponderado: {df_kpis['total_revenue'].sum() / df_kpis['total_cac'].sum():.2f}x")

## 2Ô∏è‚É£ M√âTRICAS DE CAC - NUEVA: CAC POR PR√âSTAMO

In [ ]:
# M√©tricas a nivel cliente
query_cac_cliente = """
SELECT 
    risk_segment,
    COUNT(*) as customers,
    ROUND(AVG(total_loans), 2) as avg_loans_per_customer,
    ROUND(AVG(cac_total), 2) as avg_cac_total,
    ROUND(AVG(cac_per_loan), 2) as avg_cac_per_loan
FROM main.int_customer_loan_metrics
WHERE risk_segment IN ('High Risk', 'Medium Risk', 'Low Risk')
GROUP BY risk_segment
ORDER BY customers DESC
"""

df_cac_cliente = conn.execute(query_cac_cliente).df()

print("="*80)
print("M√âTRICAS DE CAC A NIVEL CLIENTE")
print("="*80)
print(df_cac_cliente.to_string(index=False))
print("\nÔøΩÔøΩ avg_cac_per_loan = CAC Total / Total Pr√©stamos del Cliente")

# M√©tricas a nivel pr√©stamo
query_cac_prestamo = """
SELECT 
    loan_sequence_number as prestamo_num,
    COUNT(*) as loans,
    ROUND(AVG(customer_total_loans), 1) as avg_total_loans_cliente,
    ROUND(AVG(customer_cac_per_loan), 2) as avg_cac_per_loan,
    ROUND(SUM(cac), 0) as cac_asignado,
    ROUND(AVG(revenue_total), 0) as avg_revenue
FROM main.fct_loan_financials
WHERE loan_sequence_number <= 5
GROUP BY loan_sequence_number
ORDER BY loan_sequence_number
"""

df_cac_prestamo = conn.execute(query_cac_prestamo).df()

print("\n" + "="*80)
print("M√âTRICAS DE CAC A NIVEL PR√âSTAMO")
print("="*80)
print(df_cac_prestamo.to_string(index=False))
print("\n‚úÖ Verificaci√≥n: CAC solo se asigna al pr√©stamo #1")

## Ì≥ä GR√ÅFICO 1: LTV/CAC POR SEGMENTO (CORREGIDO)

In [ ]:
# Gr√°fico LTV/CAC
fig, ax = plt.subplots(figsize=(10, 6))

colors = ['#2ecc71' if x >= 5 else '#e74c3c' if x < 3 else '#f39c12' 
          for x in df_kpis['ltv_to_cac']]

bars = ax.barh(df_kpis['risk_segment'], df_kpis['ltv_to_cac'], color=colors)
ax.axvline(x=3, color='red', linestyle='--', linewidth=2, label='M√≠nimo 3x')
ax.axvline(x=5, color='green', linestyle='--', linewidth=2, alpha=0.5, label='Target 5x')

ax.set_xlabel('LTV/CAC Ratio', fontsize=12, fontweight='bold')
ax.set_title('LTV/CAC por Segmento de Riesgo (CORREGIDO)\nCAC asignado solo al pr√©stamo #1', 
             fontsize=14, fontweight='bold')
ax.legend()

for i, (idx, row) in enumerate(df_kpis.iterrows()):
    ax.text(row['ltv_to_cac'] + 0.2, i, f"{row['ltv_to_cac']:.2f}x", 
            va='center', fontweight='bold', fontsize=11)

plt.tight_layout()
plt.savefig('graficos_finales/01_ltv_cac_corregido.png', dpi=300, bbox_inches='tight')
plt.show()

print("‚úÖ Gr√°fico guardado: 01_ltv_cac_corregido.png")

## Ì≥ä GR√ÅFICO 2: PORTFOLIO MIX

In [ ]:
# Portfolio Mix
query_mix = """
SELECT 
    risk_segment,
    COUNT(*) as total_loans,
    SUM(revenue_total) as total_revenue
FROM main.fct_loan_financials
WHERE risk_segment IN ('High Risk', 'Medium Risk', 'Low Risk')
GROUP BY risk_segment
"""

df_mix = conn.execute(query_mix).df()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

colors_pie = ['#e74c3c', '#f39c12', '#2ecc71']

# Por volumen
ax1.pie(df_mix['total_loans'], labels=df_mix['risk_segment'], autopct='%1.1f%%',
        startangle=90, colors=colors_pie, textprops={'fontsize': 11, 'fontweight': 'bold'})
ax1.set_title('Portfolio Mix - Volumen de Pr√©stamos', fontweight='bold', fontsize=12)

# Por revenue
ax2.pie(df_mix['total_revenue'], labels=df_mix['risk_segment'], autopct='%1.1f%%',
        startangle=90, colors=colors_pie, textprops={'fontsize': 11, 'fontweight': 'bold'})
ax2.set_title('Portfolio Mix - Revenue', fontweight='bold', fontsize=12)

plt.tight_layout()
plt.savefig('graficos_finales/02_portfolio_mix.png', dpi=300, bbox_inches='tight')
plt.show()

print("‚úÖ Gr√°fico guardado: 02_portfolio_mix.png")

## Ì≥ä GR√ÅFICO 3: VINTAGE CURVES - DELINQUENCY

In [ ]:
# Vintage Curves
query_vintage = """
SELECT 
    vintage_month,
    months_on_book,
    delinquency_rate
FROM main.fct_vintage_curves
WHERE vintage_month IN ('2025-01-01', '2025-02-01', '2025-03-01')
ORDER BY vintage_month, months_on_book
"""

df_vintage = conn.execute(query_vintage).df()

fig, ax = plt.subplots(figsize=(12, 7))

for vintage in df_vintage['vintage_month'].unique():
    data = df_vintage[df_vintage['vintage_month'] == vintage]
    label = pd.to_datetime(vintage).strftime('%b %Y')
    ax.plot(data['months_on_book'], 
            data['delinquency_rate'] * 100,
            marker='o', linewidth=2, markersize=6, label=label)

ax.set_xlabel('Meses desde Desembolso', fontsize=12, fontweight='bold')
ax.set_ylabel('Delinquency Rate (%)', fontsize=12, fontweight='bold')
ax.set_title('Vintage Curves: Evoluci√≥n de Delinquency por Cohorte', 
             fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('graficos_finales/03_vintage_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print("‚úÖ Gr√°fico guardado: 03_vintage_curves.png")

# Calcular mejora
jan_final = df_vintage[df_vintage['vintage_month'] == '2025-01-01']['delinquency_rate'].iloc[-1]
mar_final = df_vintage[df_vintage['vintage_month'] == '2025-03-01']['delinquency_rate'].iloc[-1]
mejora = ((mar_final - jan_final) / jan_final) * 100
print(f"\nÌ≥à Mejora de Ene a Mar: {mejora:.1f}%")

## Ì≥ä GR√ÅFICO 4: CAC POR SEGMENTO

In [ ]:
# CAC por Segmento (solo pr√©stamos #1)
query_cac = """
WITH first_loans AS (
    SELECT 
        risk_segment,
        cac
    FROM main.fct_loan_financials
    WHERE loan_sequence_number = 1 
      AND cac > 0
      AND risk_segment IN ('High Risk', 'Medium Risk', 'Low Risk')
)
SELECT 
    risk_segment,
    COUNT(*) as customers,
    ROUND(AVG(cac), 2) as avg_cac
FROM first_loans
GROUP BY risk_segment
"""

df_cac = conn.execute(query_cac).df()

fig, ax = plt.subplots(figsize=(10, 6))
colors = ['#2ecc71', '#f39c12', '#e74c3c']
bars = ax.bar(df_cac['risk_segment'], df_cac['avg_cac'], color=colors)
ax.set_ylabel('CAC Promedio ($)', fontsize=12, fontweight='bold')
ax.set_title('CAC Promedio por Segmento (Solo Pr√©stamo #1 con CAC>0)', 
             fontsize=14, fontweight='bold')

for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'${height:.0f}',
            ha='center', va='bottom', fontweight='bold', fontsize=11)

plt.tight_layout()
plt.savefig('graficos_finales/04_cac_por_segmento.png', dpi=300, bbox_inches='tight')
plt.show()

print("‚úÖ Gr√°fico guardado: 04_cac_por_segmento.png")

## Ì≥ä GR√ÅFICO 5: PERFORMANCE POR RISK BAND

In [ ]:
# Performance por Risk Band
query_band = """
WITH customer_lifetime AS (
    SELECT 
        user_id,
        MAX(risk_band_production) as risk_band,
        SUM(revenue_total) as lifetime_revenue,
        SUM(cac) as total_cac,
        AVG(CASE WHEN is_delinquent = 1 THEN 1.0 ELSE 0.0 END) as delinq_rate
    FROM main.fct_loan_financials
    WHERE risk_band_production IN ('1', '2', '3', '4.1', '4.2', '5')
    GROUP BY user_id
)
SELECT 
    risk_band,
    COUNT(*) as customers,
    ROUND(SUM(lifetime_revenue) / NULLIF(SUM(total_cac), 0), 2) as ltv_cac,
    ROUND(AVG(delinq_rate) * 100, 1) as avg_delinq_pct
FROM customer_lifetime
GROUP BY risk_band
ORDER BY 
    CASE 
        WHEN risk_band = '1' THEN 1
        WHEN risk_band = '2' THEN 2
        WHEN risk_band = '3' THEN 3
        WHEN risk_band = '4.1' THEN 4
        WHEN risk_band = '4.2' THEN 5
        WHEN risk_band = '5' THEN 6
    END
"""

df_band = conn.execute(query_band).df()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# LTV/CAC por band
colors_bar = ['#e74c3c' if x < 3 else '#f39c12' if x < 5 else '#2ecc71' 
              for x in df_band['ltv_cac']]
ax1.bar(df_band['risk_band'], df_band['ltv_cac'], color=colors_bar)
ax1.axhline(y=3, color='red', linestyle='--', linewidth=2, alpha=0.7)
ax1.set_xlabel('Risk Band', fontweight='bold')
ax1.set_ylabel('LTV/CAC', fontweight='bold')
ax1.set_title('LTV/CAC por Risk Band', fontweight='bold')

# Delinquency por band
ax2.bar(df_band['risk_band'], df_band['avg_delinq_pct'], color='#3498db')
ax2.set_xlabel('Risk Band', fontweight='bold')
ax2.set_ylabel('Delinquency (%)', fontweight='bold')
ax2.set_title('Delinquency Promedio por Risk Band', fontweight='bold')

plt.tight_layout()
plt.savefig('graficos_finales/05_performance_risk_band.png', dpi=300, bbox_inches='tight')
plt.show()

print("‚úÖ Gr√°fico guardado: 05_performance_risk_band.png")

## Ì≥ä GR√ÅFICO 6: PRIMER PR√âSTAMO VS RECURRENTE

In [ ]:
# Primer vs Recurrente
query_recurrente = """
SELECT 
    CASE WHEN loan_sequence_number = 1 THEN 'Primer Pr√©stamo' ELSE 'Recurrente' END as tipo,
    COUNT(*) as loans,
    ROUND(AVG(revenue_total), 0) as avg_revenue,
    ROUND(SUM(CASE WHEN is_delinquent = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) as delinq_pct
FROM main.fct_loan_financials
GROUP BY tipo
ORDER BY tipo DESC
"""

df_recurrente = conn.execute(query_recurrente).df()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Volumen
ax1.bar(df_recurrente['tipo'], df_recurrente['loans'], color=['#3498db', '#2ecc71'])
ax1.set_ylabel('N√∫mero de Pr√©stamos', fontweight='bold')
ax1.set_title('Volumen: Primer Pr√©stamo vs Recurrente', fontweight='bold')
for i, v in enumerate(df_recurrente['loans']):
    ax1.text(i, v, f'{v:,}', ha='center', va='bottom', fontweight='bold')

# Delinquency
ax2.bar(df_recurrente['tipo'], df_recurrente['delinq_pct'], color=['#e74c3c', '#2ecc71'])
ax2.set_ylabel('Delinquency (%)', fontweight='bold')
ax2.set_title('Delinquency: Primer vs Recurrente', fontweight='bold')
for i, v in enumerate(df_recurrente['delinq_pct']):
    ax2.text(i, v, f'{v}%', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('graficos_finales/06_primer_vs_recurrente.png', dpi=300, bbox_inches='tight')
plt.show()

print("‚úÖ Gr√°fico guardado: 06_primer_vs_recurrente.png")

# C√°lculo de mejora
primer_delinq = df_recurrente[df_recurrente['tipo'] == 'Primer Pr√©stamo']['delinq_pct'].iloc[0]
recurrente_delinq = df_recurrente[df_recurrente['tipo'] == 'Recurrente']['delinq_pct'].iloc[0]
mejora_delinq = ((recurrente_delinq - primer_delinq) / primer_delinq) * 100
print(f"\nÌ≥à Mejora en delinquency (Recurrente vs Primer): {mejora_delinq:.1f}%")

## Ì≥ä GR√ÅFICO 7: HEATMAP VINTAGE √ó SEGMENTO

In [ ]:
# Heatmap Vintage x Segmento
query_heatmap = """
WITH customer_lifetime AS (
    SELECT 
        l.user_id,
        DATE_TRUNC('month', l.vintage_month) as vintage,
        MAX(l.risk_segment) as risk_segment,
        SUM(l.revenue_total) as lifetime_revenue,
        SUM(l.cac) as total_cac
    FROM main.fct_loan_financials l
    WHERE l.risk_segment IN ('High Risk', 'Medium Risk', 'Low Risk')
    GROUP BY l.user_id, vintage
)
SELECT 
    strftime(vintage, '%b') as vintage_month,
    risk_segment,
    ROUND(SUM(lifetime_revenue) / NULLIF(SUM(total_cac), 0), 2) as ltv_cac
FROM customer_lifetime
GROUP BY vintage, risk_segment
ORDER BY vintage, risk_segment
"""

df_heatmap = conn.execute(query_heatmap).df()
pivot = df_heatmap.pivot(index='risk_segment', columns='vintage_month', values='ltv_cac')

fig, ax = plt.subplots(figsize=(10, 6))
sns.heatmap(pivot, annot=True, fmt='.2f', cmap='RdYlGn', center=5,
            cbar_kws={'label': 'LTV/CAC'}, ax=ax, vmin=0, vmax=12,
            linewidths=0.5, linecolor='gray')
ax.set_title('Heatmap: LTV/CAC por Vintage √ó Segmento', 
             fontsize=14, fontweight='bold')
ax.set_xlabel('Mes de Vintage', fontweight='bold')
ax.set_ylabel('Segmento de Riesgo', fontweight='bold')

plt.tight_layout()
plt.savefig('graficos_finales/07_heatmap_vintage_segment.png', dpi=300, bbox_inches='tight')
plt.show()

print("‚úÖ Gr√°fico guardado: 07_heatmap_vintage_segment.png")

## Ì≥ä GR√ÅFICO 8: CONTRIBUTION MARGIN

In [ ]:
# Contribution Margin
query_margin = """
SELECT 
    risk_segment,
    ROUND(AVG(contribution_margin_pct) * 100, 1) as avg_margin_pct,
    COUNT(*) as loans
FROM main.fct_loan_financials
WHERE risk_segment IN ('High Risk', 'Medium Risk', 'Low Risk')
GROUP BY risk_segment
ORDER BY avg_margin_pct DESC
"""

df_margin = conn.execute(query_margin).df()

fig, ax = plt.subplots(figsize=(10, 6))
colors = ['#2ecc71' if x > 99 else '#f39c12' for x in df_margin['avg_margin_pct']]
bars = ax.bar(df_margin['risk_segment'], df_margin['avg_margin_pct'], color=colors)
ax.set_ylabel('Contribution Margin (%)', fontsize=12, fontweight='bold')
ax.set_title('Contribution Margin Promedio por Segmento', 
             fontsize=14, fontweight='bold')
ax.set_ylim(90, 100)

for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.1f}%',
            ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('graficos_finales/08_contribution_margin.png', dpi=300, bbox_inches='tight')
plt.show()

print("‚úÖ Gr√°fico guardado: 08_contribution_margin.png")

## Ì≥ä GR√ÅFICO 9: CAC AMORTIZADO POR N√öMERO DE PR√âSTAMO

In [ ]:
# Nuevo gr√°fico: CAC amortizado
fig, ax = plt.subplots(figsize=(10, 6))

colors = ['#3498db', '#2ecc71', '#27ae60', '#16a085', '#1abc9c']
bars = ax.bar(df_cac_prestamo['prestamo_num'], 
              df_cac_prestamo['avg_cac_per_loan'], 
              color=colors)

ax.set_xlabel('N√∫mero de Pr√©stamo', fontsize=12, fontweight='bold')
ax.set_ylabel('CAC Amortizado ($)', fontsize=12, fontweight='bold')
ax.set_title('CAC Amortizado por N√∫mero de Pr√©stamo\n(CAC Total / Total Pr√©stamos del Cliente)', 
             fontsize=14, fontweight='bold')

for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'${height:.2f}',
            ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('graficos_finales/09_cac_amortizado.png', dpi=300, bbox_inches='tight')
plt.show()

print("‚úÖ Gr√°fico guardado: 09_cac_amortizado.png")
print(f"\nÌ≥ä Observaci√≥n: CAC amortizado disminuye de ${df_cac_prestamo['avg_cac_per_loan'].iloc[0]:.2f} a ${df_cac_prestamo['avg_cac_per_loan'].iloc[-1]:.2f} en pr√©stamos recurrentes")

## ÌæØ CONCLUSIONES FINALES

In [ ]:
print("="*80)
print("CONCLUSIONES Y RECOMENDACIONES - PORTFOLIO Q1 2025")
print("="*80)

print("\nÌæØ HALLAZGOS CLAVE:")
print("-"*80)
print(f"""
1. LTV/CAC CORREGIDO (CAC solo en pr√©stamo #1):
   ‚≠ê High Risk: {df_kpis[df_kpis['risk_segment']=='High Risk']['ltv_to_cac'].values[0]:.2f}x (EXCELENTE)
   ‚úÖ Medium Risk: {df_kpis[df_kpis['risk_segment']=='Medium Risk']['ltv_to_cac'].values[0]:.2f}x (BUENO)
   ‚ùå Low Risk: {df_kpis[df_kpis['risk_segment']=='Low Risk']['ltv_to_cac'].values[0]:.2f}x (NO RENTABLE)

2. CLIENTES RECURRENTES:
   - {df_recurrente[df_recurrente['tipo']=='Recurrente']['loans'].values[0]:,} pr√©stamos ({df_recurrente[df_recurrente['tipo']=='Recurrente']['loans'].values[0]*100/df_recurrente['loans'].sum():.1f}% del portfolio)
   - {mejora_delinq:.1f}% MENOS delinquency vs primer pr√©stamo
   - ROI infinito (CAC = $0)

3. NUEVAS M√âTRICAS CAC:
   - CAC promedio por cliente: ${df_cac_cliente['avg_cac_total'].mean():.2f}
   - Pr√©stamos promedio por cliente: {df_cac_cliente['avg_loans_per_customer'].mean():.1f}
   - CAC amortizado: ${df_cac_cliente['avg_cac_per_loan'].mean():.2f} por pr√©stamo

4. VINTAGE CURVES:
   - Mejora de {mejora:.1f}% en delinquency (Ene‚ÜíMar)
   - Tendencia positiva en calidad de originaci√≥n
""")

print("\nÌ≤° RECOMENDACIONES ESTRAT√âGICAS:")
print("-"*80)
print("""
1. MANTENER Y ESCALAR High Risk:
   - 10.37x LTV/CAC es excepcional
   - Aumentar volumen de adquisici√≥n
   - Mantener pol√≠ticas de underwriting actuales

2. OPTIMIZAR Medium Risk:
   - Reducir CAC de $71 a $50 mejorar√° LTV/CAC a 8x+
   - Optimizar canales de adquisici√≥n
   - Foco en org√°nico y referidos

3. ELIMINAR/REDUCIR Low Risk:
   - 2.47x LTV/CAC no es sostenible
   - CAC demasiado alto ($92) vs revenue generado
   - Reasignar presupuesto a High Risk

4. MAXIMIZAR RETENCI√ìN:
   - Pr√©stamos recurrentes tienen 43% menos delinquency
   - CAC = $0 en recurrentes = ROI infinito
   - Programas de fidelizaci√≥n y cross-sell

5. CONTINUAR MONITOREO VINTAGE:
   - Mejora mensual de 31% es positiva
   - Mantener disciplina de underwriting
""")

print("\n" + "="*80)
print("FIN DEL AN√ÅLISIS")
print("="*80)

conn.close()
print("\n‚úÖ An√°lisis completado y conexi√≥n cerrada")